## Demonstrating PyMaxspots with the Umatilla aeromagnetics dataset

In this notebook, we will demonstrate how to use the **pymaxpots** package to identify max spots and create lineations from aeromagnetic data.

First we import some necessary libraries.

In [1]:
import numpy as np
import pandas as pd
import pathlib
import rasterio

Next we load the **pymaxspots** library.

In [2]:
import pymaxspots

Now we load the dataset from northeastern Oregon (Earney and others, 2022).  This dataset contains the pseudogravity transform from the aeromagnetic data.

In [ ]:
pseudograv_raster_path = pathlib.Path("../data/Umatilla_final_220308/Magnetics/Umatilla_aeromag_Pseudogravity.tif")
pseudograv_dataset = rasterio.open(pseudograv_raster_path)

RasterioIOError: data\Umatilla_final_220308\Magnetics\Umatilla_aeromag_Pseudogravity.tif: No such file or directory

Now we read the pseudogravity data from the dataset.  We need to take some additional steps to replace any "nodata" values to NAN, to ensure that these values are not used to calculate the maxspots.

In [ ]:
masked_img = pseudograv_dataset.read(1, masked=True) # get the first band from the file
pseudograv = masked_img.filled(np.nan) # replace all instances of the nodata value with np.nan

Now we get the "geotransform" which is the parameters that define the grid.  Note that by default, `dy` is usually negative, but **pymaxspots** requires that both `dx` and `dy` be positive.

In [ ]:
ulx, dx, rr, uly, cr, dy = pseudograv_dataset.get_transform()
dy_abs = np.abs(dy) # ensuring that dy is represented as a positive value

Next, we use `pymaxspots.horizontal_gradient_magnitude()` to calculate the horizontal gradient magnitude (HGM) of the pseudogravity data.

In [ ]:
hgm = pymaxspots.horizontal_gradient_magnitude(pseudograv, dx, dy_abs)

Now we are ready to use `pymaxspots.maxspots()` to calculate the maxspots:

In [ ]:
max_spots = pymaxspots.maxspots(hgm, ulx, uly, dx, dy_abs)

`pymaxspots.maxspots()` returns a structured numpy array, with each row representing a different point.  The columns are as follows:
- `ID` (str): string description of shape of curvature of max point.
- `X` (float): x-coordinate.
- `Y` (float): y-coordinate.
- `HGM` (float): value of horizontal gradient magnitude at the max spot.
- `elong` (float): elongation ratio (e1/e2).
- `strike` (float): azimuth of principle direction of elongation.
- `e1` (float): big eigenvalue.
- `e2` (float): small eigenvalue.
- `type` (int): integer number denoting curvature shape.

In [ ]:
max_spots

We can easily convert this array into a **pandas** data frame for easier data manipulation:

In [ ]:
max_spots_df = pd.DataFrame(max_spots)

Now, we will convert the data frame object `max_spots_df` into a geospatial object using **geopandas**, and save it as a shapefile.  To do this, we need the coordinate system of the data, which we will get from the HGM file. 

In [ ]:
import geopandas as gpd

crs_from_hgm = pseudograv_dataset.crs # get the coordinate system from the HGM file

max_points_gdf = gpd.GeoDataFrame(max_spots_df,
                                  geometry=gpd.points_from_xy(max_spots_df['X'],
                                  max_spots_df['Y'], crs=crs_from_hgm))

max_points_gdf.to_file("umatilla_aeromag_maxspots.shp", driver="ESRI Shapefile")

Now we will display the most interesting subset of these points on a map.

Typically we are most interested in points where the curvature shape is classified as a "ridge" because these points form linear features that can be associated with geologic structures, such as faults.

We will select the maxspots that have a shape classification of "ridge":

In [ ]:
max_spots_df_ridge = max_spots_df[max_spots_df["ID"]=="ridge"]

Now we will display these points in map view using the **matplotlib** package.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.scatter(max_spots_df_ridge["X"], max_spots_df_ridge["Y"], s=0.5)
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
ax.set_aspect("equal")

Now, we will connect these "ridge" points into lines using the `pymaxspots.maxspots_lineations()` function.

This function connects the points together into lines if they are within a maximum distance of each other (set by the `dist_tol` parameter), and if each subsequent line segment is within a maximum azimuth angle (set by the `azimuth_tol` parameter) of the previous segment.  Lines must have a minimum number of points (set by the `min_line_segments` parameter).

The user does not need to input any of these parameters, as they will be set by default, but we will demonstrate how to use them here.

First, we can assign the maximum distance tolerance as 1.75 times the the mean distance between nearest neighbor pairs.

In [ ]:
mean_nn_dist = pymaxspots.mean_nearest_neighbor_distance(max_spots_df_ridge["X"], max_spots_df_ridge["Y"])
dist_tol = 1.75 * mean_nn_dist
print("dist_tol = {}".format(dist_tol))

Let's now use `pymaxspots.maxspots_lineations()` to connect the points into lines.  We will use the previously calculated distance tolerance, and we will set an azimuth tolerance of 40 degrees and set the minimum number of line segments to 3.

In [ ]:
# find the lineations
lines = pymaxspots.maxspots_lineations(max_spots_df_ridge["X"], max_spots_df_ridge["Y"], dist_tol=dist_tol, azimuth_tol=40, min_line_segments=3)

Note that `pymaxspots.maxspots_lineations()` returns a list of "lines".  Each "line" is a list of the indices of the `x, y` coordinates, ordered in the sequence in which they are connected.

We can obtain the connected sequence of coordinates by using these indices.  By using the **shapely** library, we can convert each line into a vector line feature.

In [ ]:
from shapely.geometry import LineString

X = np.c_[max_spots_df_ridge["X"], max_spots_df_ridge["Y"]]
lines_xy = [LineString([X[i] for i in line]) for line in lines]

We can also save these line features as a shapefile using **geopandas**.

In [ ]:
lines_xy_gdf = gpd.GeoDataFrame(geometry=lines_xy, crs=crs_from_hgm)
lines_xy_gdf.to_file("umatilla_aeromag_lineations.shp", driver="ESRI Shapefile")

We can also display the line features in map view using **matplotlib**:

In [ ]:
fig, ax = plt.subplots()

lines_xy_gdf.plot(ax=ax, lw=0.5, color="red")
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
ax.set_aspect("equal")